<a href="https://colab.research.google.com/github/sabarish244/Face-Recognition-using-Transfer-learning/blob/master/transfer_learning_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications import InceptionResNetV2

#model needs 224x224 pixels as input
img_rows,img_cols= 224,224

# Re-loads the model without the top or FC layers
model = InceptionResNetV2(weights='imagenet'  , include_top = False ,input_shape=(img_rows,img_cols,3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False

    # Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [0]:
def addTopModelmodel(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = Flatten(name='flatten')(top_model)
    top_model = Dense(4096,activation='relu')(top_model)
    top_model = Dense(4096,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 2
num_classes = 4

FC_Head = addTopModelmodel(model, num_classes)

newmodel = Model(inputs = model.input, outputs = FC_Head)

print(newmodel.summary())

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/drive/My Drive/Sabari/train_set'
test_data_dir = '/content/drive/My Drive/Sabari/test_set'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
test_datagen = ImageDataGenerator(rescale=1./255)
 

 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols))
 
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows, img_cols))

In [0]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("/content/drive/My Drive/face_recog.h5",
                             monitor="val_accuracy",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_accuracy',
                          verbose=1, 
                          patience=5,
                          restore_best_weights=True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
newmodel.compile(loss = 'binary_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

history = newmodel.fit_generator(
    generator=train_generator,
    epochs = 10,
    callbacks = callbacks,
    validation_data = test_generator)
newmodel.save("/content/drive/My Drive/face_recog.h5")

In [0]:
from keras.models import load_model
from keras.applications import decode_predictions
from keras.preprocessing import image
his_img = image.load_img("/content/drive/My Drive/test.jpg", target_size=(224,224))
import numpy as np
his_img = image.img_to_array(his_img)
exp_img = np.expand_dims(his_img, axis=0)
model = load_model("/content/drive/My Drive/face_recog.h5")
pred = model.predict(exp_img)
pred